In [0]:
dbutils.widgets.text("p_environment", "")
dbutils.widgets.text("p_internal_product_id", "")
dbutils.widgets.text("p_source_server_name", "")
dbutils.widgets.text("p_source_database_name", "")
dbutils.widgets.text("p_internal_client_id", "")
dbutils.widgets.text("p_internal_facility_id", "")
dbutils.widgets.text("p_ingestion_pipeline_name", "")

In [0]:
%run ../../classhandlers/UtilityHandler

In [0]:
%run ../../classhandlers/ProcessConfigHandler

In [0]:
required_parameters = ["p_environment", "p_internal_product_id", "p_ingestion_pipeline_name"]
run_parameters_dict = NotebookUtils.get_parameters_dict(dbutils.notebook.entry_point.getCurrentBindings())

NotebookUtils.validate_parameters(run_parameters_dict, required_parameters)

#convert to int
run_parameters_dict["p_internal_product_id"] = int(run_parameters_dict["p_internal_product_id"])
run_parameters_dict["p_internal_client_id"] = int(run_parameters_dict["p_internal_client_id"]) if run_parameters_dict["p_internal_client_id"] else None
run_parameters_dict["p_internal_facility_id"] = int(run_parameters_dict["p_internal_facility_id"]) if run_parameters_dict["p_internal_facility_id"] else None

print(f"Run Parameters:\n{run_parameters_dict}")

In [0]:
import json
from pyspark.sql.functions import col, trim

In [0]:
process_config = ProcessConfigData(run_parameters_dict["p_environment"])

v_unity_catalog = process_config.get_config_attribute_value('AnalyticsUnityCatalog')
v_managed_location_root_path = process_config.get_config_attribute_value('AdlsAnalyticsFullpathUri')

df_config_data_row = process_config.get_server_list (
                                                        run_parameters_dict["p_internal_product_id"], 
                                                        run_parameters_dict["p_source_server_name"], 
                                                        run_parameters_dict["p_source_database_name"], 
                                                        run_parameters_dict["p_internal_client_id"], 
                                                        run_parameters_dict["p_internal_facility_id"],
                                                        run_parameters_dict["p_ingestion_pipeline_name"]
                                                    )

config_data_row_dict = [row.asDict() for row in df_config_data_row.collect()][0]
config_data_row_dict['Environment'] = run_parameters_dict["p_environment"]
config_data_row_dict['Status'] = 'Pending'
#config_data_row_dict['DestinationCatalog'] = v_unity_catalog
#config_data_row_dict['ManagedLocationRootPath'] = v_managed_location_root_path
config_data_row_dict['ProcessIdentifier'] = f"Pipeline={config_data_row_dict['GatewayPipelineName']} Server={config_data_row_dict['SourceServerName1']} Database={config_data_row_dict['SourceDatabaseName1']}"

dbutils.jobs.taskValues.set(key = run_parameters_dict["p_ingestion_pipeline_name"], value = config_data_row_dict)

print(f"key = {run_parameters_dict["p_ingestion_pipeline_name"]}, value = {config_data_row_dict}")